# **Leveraging LLMs for Text Generation and Summarization**


# **Table of Contents**

- [1.1 Introduction & Motivation](#11-introduction--motivation)
- [1.2 Transformer Architecture Fundamentals](#12-transformer-architecture-fundamentals)
- [1.3 Categories of LLMs](#13-categories-of-llms)
- [1.4 Basic Text Generation](#14-basic-text-generation)
- [2.1 Extractive Summarization](#21-extractive-summarization)
- [2.2 Abstractive Summarization & Control Parameters](#22-abstractive-summarization--control-parameters)
- [2.3 Evaluation Metrics for Summarization](#23-evaluation-metrics-for-summarization)
- [3.1 Complex Document Handling](#31-complex-document-handling)
- [3.2 Multi-Document Summarization](#32-multi-document-summarization)
- [3.3 Multimodal Summarization](#33-multimodal-summarization)
- [Practical Exercise: Building Your Custom Summarization System](#practical-exercise-building-your-custom-summarization-system)

# **PART I - Foundations**

# **1.1 Introduction & Motivation**
### Why Summarization Matters
- Information overload in the digital age
- Business applications: news aggregation, document processing, research synthesis
- Personal productivity: email summaries, meeting notes, research papers
- Accessibility: making complex content digestible

### The Evolution Story
- Traditional approaches (pre-2020):
  - Manual feature engineering (TF-IDF, sentence scoring)
  - Task-specific model training
  - Domain-specific datasets
  - Complex evaluation pipelines

- The LLM Revolution (post-2020):
  - Simple prompt: "Summarize this article in 3 sentences"
  - Often outperforms specialized models
  - Research finding: Human evaluators prefer GPT-3 summaries 77% of the time

By the end of Part I, participants should be able to:

- Explain why LLMs revolutionized summarization
- Choose appropriate LLM architectures for different tasks
- Control basic text generation parameters
- Engineer effective prompts for structured output
- Generate their first LLM-based summary
- Understand the foundation for advanced techniques

# **1.2 Transformer Architecture Fundamentals**

## The Transformer Architecture

The transformer architecture revolutionized NLP when introduced in the paper "Attention is All You Need" (Vaswanathan et al., 2017).

<!-- Transformer Architecture -->
<div align="center">
<img src='images/transformer architecture.png' alt="Transformer architecture" width=1000 height=700>
</div>


### Key Components:
- **Self-Attention Mechanism**: Allows the model to weigh the importance of different words in context
- **Multi-Head Attention**: Parallel attention mechanisms capturing different relationships
- **Positional Encoding**: Helps the model understand word order
- **Feed-Forward Networks**: Process the representations from attention layers
- **Layer Normalization**: Stabilizes training


### Why Transformers Excel at Text Tasks,
1. **Parallel Processing**: Unlike RNNs, can process entire sequences simultaneously
2. **Long-Range Dependencies**: Attention mechanism captures distant relationships
3. **Context Awareness**: Each token attends to all other tokens in the sequence
4. **Scalability**: Architecture scales well with data and compute

## **1.3 Categories of LLMs**

LLMs come in different architectural variants, each with strengths for different tasks:

### 1. Decoder-Only Models (Autoregressive)
- Examples: GPT series, LLaMA, Claude
**When to use:**
- Creative, narrative summaries
- When you need natural, flowing text
- General-purpose summarization
- When you have good prompting capabilities

**Practical considerations:**
- May hallucinate or add information
- Require careful prompt engineering
- Good for varying summary styles
- Cost-effective for simple tasks

**Example use cases:**
- Blog post summaries
- Creative writing summaries
- Social media content

### 2. Encoder-Only Models
- Examples: BERT, RoBERTa
**When to use:**
- Extractive summarization
- When you need to understand document context
- Sentence ranking and selection
- Classification-based approaches

**Practical considerations:**
- Limited generation capabilities
- Excellent for understanding
- Good for factual accuracy
- Computationally efficient

**Example use cases:**
- News article key sentence extraction
- Legal document important clause identification

### 3. Encoder-Decoder Models
**When to use:**
- Balanced understanding and generation
- Abstractive summarization
- When you need both comprehension and creativity
- Formal summarization tasks

**Practical considerations:**
- Higher computational requirements
- Best for abstractive tasks
- Good balance of accuracy and fluency
- Requires more training data

**Example use cases:**
- Academic paper abstracts
- Technical documentation summaries
- Multi-document synthesis


<img src='images/llm_categories.png'>

# **1.4 Basic Text Generation**



#### Understanding Generation Parameters

- **Temperature (0.0-2.0)**: Controls randomness
  - 0.0-0.3: Deterministic, factual content
  - 0.4-0.7: Balanced creativity and coherence  
  - 0.8-1.2: Creative, varied output
  - 1.3+: Highly creative but potentially incoherent

- **Top-p (0.0-1.0)**: Nucleus sampling
  - Selects the most probable tokens whose cumulative probability exceeds a certain threshold p
  - Lower values: More focused, consistent
  - Higher values: More diverse vocabulary

- **Top-k**: Limits vocabulary to k most likely tokens and samples from it
  - Lower values: More predictable
  - Higher values: More creative word choices

- **Beam-Search**
<img src='images/beam-search.jpg' width=900 height=431>
  -  Sequence score is cumulative sum of the log probability of every token in the beam.

#### Hands-on Parameter Experimentation

In [ ]:
# Set up the environment
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    from google.colab import userdata
    api_key = userdata.get('OPENROUTER_API_KEY')
else:
  from dotenv import load_dotenv
  load_dotenv()

import os
import dotenv
from dotenv import load_dotenv
from openai import OpenAI
import json
from typing import List
load_dotenv()

In [22]:
class LLMClient:

    def __init__(self, model_name="google/gemini-2.0-flash-lite-001"):
        self.model = model_name
        self.api_key = os.getenv("OPENROUTER_API_KEY")
        self.client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=self.api_key)

    def generate(self, text, temperature=0.8, max_tokens=2000, tools=None):
        """Generate text using autoregressive model"""
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "user", "content": text}
                ],
                temperature=temperature,
                max_tokens=max_tokens,
                model=self.model,
                tool_choice="auto",
                tools=tools,
                extra_body={},
            )
            # Extract and return the response
            result = response.choices[0].message.content
            return result
        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
def compare_parameters(prompt: str, temperatures: List[float], client: LLMClient):

    results = []
    for temperature in temperatures:
        result = client.generate(prompt, temperature=temperature)
        results.append(
            {
                "temperature": temperature,
                "response": result,
                "length": len(result.split())
            }
        )   
    return results


test_prompt = "Write a creative opening sentence for a science fiction story about time travel."
temperatures = [0.1, 0.5, 0.9, 1.2]

results = compare_parameters(test_prompt, temperatures, LLMClient()),
for result in results[0]:
    print(f"Temperature: {result['temperature']}"),
    print(f"Response: {result['response']}"),
    print(f"Word count: {result['length']}"),
    print("-" * 40)


#### Prompt Engineering Fundamentals


<img src="images/co-star.png" width=700 height=500>



| Element       | Description                              |
| ------------- | ---------------------------------------- |
| **C**ontext   | Provide background information           |
| **O**bjective | State the goal of the task               |
| **S**tyle     | Specify tone, format, or constraints     |
| **T**ask      | What the model should actually do        |
| **A**udience  | Who the output is intended for           |
| **R**esponse  | Clarify what the output should look like |

**Prompt Example:**

*Context*: You are a career advisor writing content for a university’s job preparation website. Many students are unsure how to describe their achievements on resumes, particularly in action-result format.

*Objective*: Help students craft clear and impressive resume bullet points for internships in data science.

*Style*: Keep the tone professional and concise. Use strong action verbs and quantify results wherever possible. Avoid first-person language.

*Task*: Based on the details provided, generate 3 resume bullet points that follow best practices in resume writing.

*Audience*: Undergraduate students applying for internships in data science roles.

*Response*: Your output should be a bulleted list of exactly 3 resume-ready statements.

## Structured output generation

Getting LLMs to produce consistent, parseable output formats is crucial for integration with downstream systems.

### Prompt based structured generation

In [ ]:
schema = {
    "name": "string",
    "age": "integer",
    "profession": "string",
    "language": "string",
    "origin": "string"
}

character_bio = """
"Dr. Amara Liu is a 35-year-old astrophysicist from Shanghai. She researches dark matter and enjoys stargazing and sketching. Fluent in Mandarin and English."
"""

prompt = (
    f"{character_bio}\n\n"
    "Extract the following structured data as JSON:\n"
    f"{json.dumps(schema, indent=2)}\n\n"
    "Respond ONLY with valid JSON that matches the schema above."
)
llm_model = LLMClient()
response = llm_model.generate(prompt)

# Try to extract JSON
try:
    structured_data = json.loads(response)
except json.JSONDecodeError:
    import re
    match = re.search(r'\{.*\}', response, re.DOTALL)
    structured_data = json.loads(match.group()) if match else {"error": "invalid format"}

print(structured_data)


### Schema Aware structured Output

In [ ]:
# Structure Output via function calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_profile",
            "description": "Extracts a person's profile.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "age": {"type": "integer"},
                    "profession": {"type": "string"},
                    "language": {"type": "string"},
                    "origin": {"type": "string"}
                },
                "required": ["name", "age", "profession", "language", "origin"]
            }
        }
    }
]

prompt = f"What is the name, age, profession, language, and origin of the person in the following text:\n {character_bio}"
response = llm_model.generate(prompt, tools=tools)
print(response.choices[0].message.tool_calls[0].function.arguments)


## Knowledge Check - Can you explain/do the following?

### Conceptual Understanding
- [ ] **Why did LLMs revolutionize summarization?**
  - What changed from pre-2020 to post-2020 approaches?
  - Why do simple prompts often outperform specialized models?

- [ ] **Architecture Selection**
  - When would you choose a decoder-only model (like GPT) for summarization?
  - When would you choose an encoder-decoder model (like BART/T5)?
  - What are the trade-offs between them?

- [ ] **Parameter Control**
  - How does temperature affect summary creativity vs. consistency?
  - What's the difference between top-p and top-k sampling?
  - When would you use low vs. high temperature for summarization?

### Practical Skills
- [ ] **Prompt Engineering**
  - Can you structure a prompt using the CO-STAR framework?
  - How would you modify a prompt to get a more focused summary?
  - What would you do if your summary is too generic?

- [ ] **Structured Output**
  - Can you design a JSON schema for extracting key information?
  - When would you use function calling vs. prompt-based structured generation?

## Bridge to Part II: Prediction Exercise

Before we dive into formal summarization techniques, make a prediction:

**Scenario:** You need to summarize a 1000-word news article about a scientific breakthrough.

**Question:** Which approach do you think will work better and why?

A) **Extractive Approach**: Select the most important sentences from the original article
B) **Abstractive Approach**: Generate a new summary using an LLM

# **Part II - Core Summarization Techniques**

By the end of Part II, participants should be able to:

- Implement both extractive and abstractive summarization
- Choose between zero-shot and few-shot approaches based on task requirements
- Evaluate summary quality using multiple metrics
- Understand the trade-offs between different summarization approaches
- Apply domain-specific prompting strategies
- Interpret evaluation results to improve summarization systems

<img src="images/text summarization.jpg" width=1000>

## **2.1 Extractive Summarization**

Extractive summarization selects the most important sentences from the orginal text to create the summary.

## The Basic Process:
1. Score sentences based on importance
2. Select top-scoring sentences using ranking algorithm (TF IDF, SVM, etc)
3. Arrange in coherent order (usually original order)

## Advantages:
- Factually accurate (uses original text)
- Computationally efficient
- Works well for objective content

## Disadvantages:
- May be disconnected or redundant
- Cannot reformulate or simplify complex content
- Limited by quality of source material

## Understanding TF-IDF for Extractive Summarization

**TF-IDF (Term Frequency-Inverse Document Frequency)** is a numerical statistic that reflects how important a word is to a document within a collection of documents. It's widely used in information retrieval and text mining, making it perfect for identifying the most important sentences in a document for summarization.

## The Mathematical Foundation

### Term Frequency (TF)
**Definition**: How frequently a word appears in a specific sentence/document.

$$TF(word, sentence) = \frac{\text{Number of times word appears in sentence}}{\text{Total number of words in sentence}}$$

**Example**: In the sentence "The cat sat on the mat", the word "the" has TF = 2/6 = 0.33

### Inverse Document Frequency (IDF)
**Definition**: How rare or common a word is across all sentences in the document.

$$IDF(word) = \log\left(\frac{\text{Total number of sentences}}{\text{Number of sentences containing the word}}\right)$$

**Intuition**: 
- Common words (like "the", "and") appear in many sentences → Low IDF → Less important
- Rare words (like "quantum", "photosynthesis") appear in few sentences → High IDF → More important

### TF-IDF Score
**Final Formula**:
$$TF\text{-}IDF(word, sentence) = TF(word, sentence) \times IDF(word)$$

**Sentence Score**: Sum of TF-IDF scores for all words in the sentence
$$Score(sentence) = \sum_{word \in sentence} TF\text{-}IDF(word, sentence)$$

## Visual Example

Let's work through a simple example:

**Document**: 
- Sentence 1: "The cat sat on the mat"
- Sentence 2: "The dog ran quickly"  
- Sentence 3: "Cats and dogs are pets"

### Step 1: Calculate TF for each word in Sentence 1

| Word | Count in S1 | Total words in S1 | TF |
|------|-------------|-------------------|-----|
| the  | 2           | 6                 | 0.33|
| cat  | 1           | 6                 | 0.17|
| sat  | 1           | 6                 | 0.17|
| on   | 1           | 6                 | 0.17|
| mat  | 1           | 6                 | 0.17|

### Step 2: Calculate IDF for each word

| Word | Sentences containing word | Total sentences | IDF |
|------|--------------------------|----------------|-----|
| the  | 2 (S1, S2)              | 3              | log(3/2) = 0.18|
| cat  | 1 (S1)                  | 3              | log(3/1) = 0.48|
| sat  | 1 (S1)                  | 3              | log(3/1) = 0.48|

### Step 3: Calculate TF-IDF scores

| Word | TF   | IDF  | TF-IDF |
|------|------|------|--------|
| the  | 0.33 | 0.18 | 0.06   |
| cat  | 0.17 | 0.48 | 0.08   |
| sat  | 0.17 | 0.48 | 0.08   |

**Sentence 1 Score** = 0.06 + 0.08 + 0.08 + ... = Final Score

### Why TF-IDF Works for Summarization

### 1. **Identifies Content Words**
- Function words ("the", "and", "is") get low scores
- Content words ("algorithm", "photosynthesis", "democracy") get high scores

### 2. **Balances Frequency and Rarity**
- A word mentioned often in one sentence (high TF) but rare overall (high IDF) = very important
- A word mentioned everywhere (low IDF) = less distinctive

### 3. **Context Awareness**
- Same word gets different importance scores in different documents
- Adapts to the specific content being summarized

### Issues with TF-IDF
1. 
- **Problem**: TF-IDF ignores where sentences appear in the document.
- **Solution**: Add position-based scoring.

2. 
- **Problem**: Longer sentences automatically get higher TF-IDF scores.
- **Solution**: Normalize by optimal sentence length.

3. 
- **Problem**: Multiple sentences might convey the same information.
- **Solution**: Remove semantically similar sentences.

Hands on Implementation

In [ ]:
import nltk
nltk.download('punkt') 
nltk.download('stopwords') 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

class ExtractiveSummarizer:
    """Enhanced extractive summarizer using proper TF-IDF and additional features"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess_text(self, text):
        """Clean and prepare text for processing"""
        # Remove extra whitespace and normalize
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def extract_sentences(self, text):
        """Extract and clean sentences"""
        sentences = sent_tokenize(text)
        # Filter out very short sentences (less than 5 words)
        sentences = [s for s in sentences if len(s.split()) >= 5]
        return sentences
    
    def calculate_tfidf_scores(self, sentences):
        """Calculate TF-IDF scores for sentences"""
        # Use sklearn's TfidfVectorizer for proper TF-IDF calculation
        vectorizer = TfidfVectorizer(
            stop_words='english',
            lowercase=True,
            max_features=1000,
            ngram_range=(1, 2)  # Include bigrams for better context
        )
        
        try:
            tfidf_matrix = vectorizer.fit_transform(sentences)
            # Sum TF-IDF scores for each sentence
            sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
            return sentence_scores
        except ValueError:
            print('failed to get tf-idf')
            # Fallback to simple word frequency if TF-IDF fails

    def calculate_position_scores(self, sentences):
        """Calculate position-based scores (first and last sentences are important)"""
        num_sentences = len(sentences)
        position_scores = np.zeros(num_sentences)
        
        for i in range(num_sentences):
            if i == 0:  # First sentence
                position_scores[i] = 0.3
            elif i == num_sentences - 1:  # Last sentence
                position_scores[i] = 0.2
            elif i < num_sentences * 0.1:  # Early sentences
                position_scores[i] = 0.15
            elif i > num_sentences * 0.9:  # Late sentences
                position_scores[i] = 0.1
            else:
                position_scores[i] = 0.05
        
        return position_scores
                
    def calculate_sentence_length_scores(self, sentences):
        """Normalize scores by sentence length to avoid bias toward longer sentences"""
        length_scores = []
        for sentence in sentences:
            words = sentence.split()
            # Optimal sentence length is around 15-25 words
            if 15 <= len(words) <= 25:
                length_scores.append(1.0)
            elif 10 <= len(words) < 15 or 25 < len(words) <= 35:
                length_scores.append(0.8)
            else:
                length_scores.append(0.6)
        
        return np.array(length_scores)

    def remove_redundant_sentences(self, sentences, selected_indices, threshold=0.7):
        """Remove sentences that are too similar to already selected ones"""
        if len(selected_indices) <= 1:
            return selected_indices
        
        # Use simple word overlap for similarity
        filtered_indices = [selected_indices[0]]  # Keep the first (highest scoring)
        
        for idx in selected_indices[1:]:
            current_sentence = sentences[idx]
            current_words = set(word.lower() for word in word_tokenize(current_sentence)
                              if word.lower() not in self.stop_words and word.isalnum())
            
            is_redundant = False
            for selected_idx in filtered_indices:
                selected_sentence = sentences[selected_idx]
                selected_words = set(word.lower() for word in word_tokenize(selected_sentence)
                                   if word.lower() not in self.stop_words and word.isalnum())
                
                # Calculate Jaccard similarity
                if len(current_words) > 0 and len(selected_words) > 0:
                    intersection = current_words.intersection(selected_words)
                    union = current_words.union(selected_words)
                    similarity = len(intersection) / len(union)
                    
                    if similarity > threshold:
                        is_redundant = True
                        break
            
            if not is_redundant:
                filtered_indices.append(idx)
        
        return filtered_indices
    
    def summarize(self, text, ratio=0.3, max_sentences=None):
        """
        Generate extractive summary using improved TF-IDF approach
        
        Args:
            text: Input text to summarize
            ratio: Proportion of sentences to include (0.1 to 0.5)
            max_sentences: Maximum number of sentences (overrides ratio if specified)
        """
        # Preprocess and extract sentences
        text = self.preprocess_text(text)
        sentences = self.extract_sentences(text)
        
        if len(sentences) <= 2:
            return ' '.join(sentences)
        
        # Calculate different scoring components
        tfidf_scores = self.calculate_tfidf_scores(sentences)
        position_scores = self.calculate_position_scores(sentences)
        length_scores = self.calculate_sentence_length_scores(sentences)
        
        # Normalize TF-IDF scores to 0-1 range
        if tfidf_scores.max() > 0:
            tfidf_scores = tfidf_scores / tfidf_scores.max()
        
        # Combine scores with weights
        final_scores = (
            0.7 * tfidf_scores +      # Content importance (70%)
            0.2 * position_scores +   # Position importance (20%)
            0.1 * length_scores       # Length preference (10%)
        )
        
        # Determine number of sentences to select
        if max_sentences:
            num_sentences = min(max_sentences, len(sentences))
        else:
            num_sentences = max(1, int(len(sentences) * ratio))
        
        # Select top sentences
        top_indices = np.argsort(final_scores)[-num_sentences:][::-1]
        
        # Remove redundant sentences
        filtered_indices = self.remove_redundant_sentences(sentences, top_indices)
        
        # Sort by original order in text
        filtered_indices.sort()
        
        # Create summary
        summary_sentences = [sentences[i] for i in filtered_indices]
        summary = ' '.join(summary_sentences)
        
        return summary, {
            'selected_indices': filtered_indices,
            'scores': final_scores,
            'num_original_sentences': len(sentences),
            'num_selected_sentences': len(filtered_indices)
        }

In [ ]:
# Example usage

# Load an article
with open('articles/article.txt', 'r', encoding='utf-8') as f:
    article = f.read()
    
# Print article length
print(f"Article contains {len(sent_tokenize(article))} sentences and {len(article.split())} words")

extractive_summarizer = ExtractiveSummarizer()
generated_summary= extractive_summarizer.summarize(article, ratio=0.3)
print(f"\nExtractive Summary\n{generated_summary}")

## **2.2 Abstractive Summarization & Control Parameters**
By the end of Part II, participants should be able to:

Implement both extractive and abstractive summarization
Choose between zero-shot and few-shot approaches based on task requirements
Evaluate summary quality using multiple metrics
Understand the trade-offs between different summarization approaches
Apply domain-specific prompting strategies
Interpret evaluation results to improve summarization systems

## What is Abstractive Summarization?

Abstractive summarization involves:
- Understanding the source content deeply
- Identifying key concepts and relationships
- Generating new text by paraphrasing existing text
- Condensing information in ways that extraction cannot


In [ ]:
class AbstractiveSummarizer(LLMClient): 
    """Specialized class for abstractive summarization techniques"""
    
    def zero_shot_summarize(self, text, instruction="Summarize the following text:",
                            length=None, style=None, focus=None, audience=None):
        """Enhanced zero-shot with summarization-specific improvements"""
        # Build enhanced prompt with controls
        prompt_parts = []
        
        # Base instruction
        if audience:
            prompt_parts.append(f"You are writing for {audience}.")
        
        # Style control
        if style:
            style_modifiers = {
                "formal": "Write in a formal, professional tone.",
                "casual": "Use a conversational, accessible tone.",
                "technical": "Use precise technical language.",
                "journalistic": "Write in news article style."
            }
            prompt_parts.append(style_modifiers.get(style, ""))
        
        # Length control
        if length:
            length_modifiers = {
                "brief": "Keep it to 2-3 sentences maximum.",
                "medium": "Aim for 4-6 sentences.",
                "detailed": "Provide 8-10 sentences with comprehensive coverage."
            }
            prompt_parts.append(length_modifiers.get(length, ""))
        
        # Focus control
        if focus:
            prompt_parts.append(f"Pay special attention to: {', '.join(focus)}.")
        
        # Combine all parts
        enhanced_instruction = " ".join(prompt_parts) + f" {instruction}"
        
        prompt = f"{enhanced_instruction}\n\n{text}\n\nSummary:"
        return self.generate(prompt)
    
    def few_shot_summarize(self, text, examples):
        """Few-shot summarization with examples"""
        prompt = "Here are examples of good summaries:\n\n"
        
        for i, example in enumerate(examples, 1):
            prompt += f"Example {i}:\n"
            prompt += f"Text: {example['text']}\n"
            prompt += f"Summary: {example['summary']}\n\n"
        
        prompt += f"Now summarize this text:\n{text}\n\nSummary:"
        return self.generate(prompt)
        
    def compare_few_shot_vs_zero_shot(self, text, examples):
        """Compare zero-shot and few-shot performance"""
        
        # Zero-shot
        zero_shot = self.zero_shot_summarize(text)
        
        # Few-shot
        few_shot = self.few_shot_summarize(text, examples)
        
        print("ZERO-SHOT SUMMARY:")
        print(zero_shot)
        print(f"Length: {len(zero_shot.split())} words\n")
        
        print("FEW-SHOT SUMMARY:")
        print(few_shot)
        print(f"Length: {len(few_shot.split())} words\n")
        
        return zero_shot, few_shot

In [ ]:
#few-shot examples for different domains
news_examples = [
    {
        "text": "Scientists at MIT have developed a new battery technology that could revolutionize electric vehicles. The lithium-metal batteries can charge to 80% capacity in just 10 minutes and last for over 500,000 miles. The breakthrough addresses two major concerns about electric vehicles: charging time and battery longevity. Commercial production is expected to begin in 2026.",
        "summary": "MIT scientists created fast-charging, long-lasting batteries for electric vehicles, addressing key concerns about charging time and durability, with commercial production planned for 2026."
    },
    {
        "text": "The Federal Reserve announced a 0.25% interest rate cut today, citing concerns about slowing economic growth and inflation falling below target levels. This marks the third rate cut this year. Stock markets responded positively, with the S&P 500 gaining 2.1% in after-hours trading. Economists predict this could stimulate business investment and consumer spending.",
        "summary": "The Federal Reserve cut interest rates by 0.25% due to economic concerns, prompting positive market reactions and expectations of increased business and consumer activity."
    }
]

research_examples = [
    {
        "text": "This study examined the effects of meditation on stress hormones in 200 participants over 8 weeks. Participants who meditated daily showed a 23% reduction in cortisol levels compared to the control group. The research also found improvements in sleep quality and self-reported well-being. These findings suggest meditation could be an effective intervention for stress management.",
        "summary": "An 8-week study of 200 participants found daily meditation reduced stress hormone levels by 23% and improved sleep and well-being, supporting meditation as a stress management tool."
    }
]


# Test with a new research article
new_research = """
A comprehensive analysis of social media usage patterns among teenagers reveals concerning trends in mental health outcomes. The study, following 1,500 participants aged 13-18 over two years, found that teens spending more than 3 hours daily on social platforms showed increased rates of anxiety and depression. Particularly concerning was the correlation between late-night social media use and sleep disorders. However, the research also identified positive outcomes, including enhanced social connections and access to mental health resources. The researchers recommend implementing digital wellness programs in schools and encouraging mindful social media usage rather than complete avoidance.
"""
abstractive_summarizer = AbstractiveSummarizer()
zero_shot_result, few_shot_result = abstractive_summarizer.compare_few_shot_vs_zero_shot(new_research, research_examples)

###  Discussion Point
After running the above experiment:
1. Which summary feels more consistent with the examples provided?
2. How did the few-shot examples influence the style and content selection?
3. What are the trade-offs between zero-shot (faster) and few-shot (more controlled)?

### Advanced Prompt Engineering for Summarization

When dealing with complex documents with multiple topics or intricate relationships, breaking down the summarization process helps LLMs produce better results.

In [ ]:
class AdvancedSummarizationPrompting:
    """Advanced prompting strategies specifically for summarization"""
    
    def __init__(self):
        self.prompt_templates = self._load_templates()
        self.quality_checks = self._load_quality_prompts()
    
    def _load_templates(self):
        """Summarization-specific prompt templates"""
        return {
            "news_summary": {
                "template": """
                Context: You are a professional news editor creating summaries for busy readers.
                Objective: Create a concise summary that captures the key facts and implications.
                Style: Objective, factual, following inverted pyramid structure.
                Task: Summarize the following news article.
                Audience: General public seeking quick news updates.
                Response: Provide a 3-sentence summary focusing on who, what, when, where, why.
                
                Article: {text}
                
                News Summary:
                """,
                "parameters": {"temperature": 0.2, "top_p": 0.8}
            },
            
            "research_synthesis": {
                "template": """
                Context: You are a research analyst synthesizing academic content.
                Objective: Extract key findings and methodological insights.
                Style: Academic but accessible, emphasizing evidence and conclusions.
                Task: Summarize this research for {audience}.
                Audience: {audience}
                Response: Structure as Background, Methods, Results, Implications.
                
                Research: {text}
                
                Research Summary:
                """,
                "parameters": {"temperature": 0.3, "top_p": 0.85}
            },
            
            "meeting_minutes": {
                "template": """
                Context: You are an executive assistant creating actionable meeting summaries.
                Objective: Capture decisions, action items, and next steps.
                Style: Structured, action-oriented, clear responsibilities.
                Task: Convert this meeting transcript into organized minutes.
                Audience: Meeting participants and stakeholders.
                Response: Format as Decisions Made, Action Items, Discussion Points, Next Steps.
                
                Transcript: {text}
                
                Meeting Minutes:
                """,
                "parameters": {"temperature": 0.1, "top_p": 0.7}
            }
        }
    
    def _load_quality_prompts(self):
        """Quality control and refinement prompts"""
        return {
            "factuality_check": """
            Review this summary for factual accuracy against the source:
            Source: {source}
            Summary: {summary}
            
            Check for:
            1. Factual errors or misrepresentations
            2. Added information not in source
            3. Missing critical information
            
            Factuality Assessment:
            """,
            
            "coherence_improvement": """
            Improve the coherence and flow of this summary:
            Original: {summary}
            
            Make it more coherent by:
            1. Better transitions between ideas
            2. Logical information ordering
            3. Clearer connections between points
            
            Improved Summary:
            """,
            
            "audience_adaptation": """
            Adapt this summary for a {target_audience} audience:
            Original: {summary}
            Current audience: {current_audience}
            Target audience: {target_audience}
            
            Adapted Summary:
            """
        }
    
    def generate_domain_summary(self, text, domain="news", audience="general", **kwargs):
        """Generate summary using domain-specific prompting"""
        template = self.prompt_templates.get(domain, self.prompt_templates["news_summary"])
        
        prompt = template["template"].format(
            text=text, 
            audience=audience,
            **kwargs
        )
        
        # Use domain-specific parameters
        params = template["parameters"]
        return self.generate(prompt, **params)
    
    def multi_step_summarization(self, text, steps=None):
        """Chain-of-thought summarization for complex documents"""
        if steps is None:
            steps = ["identify_topics", "extract_key_points", "synthesize"]
        
        results = {}
        
        # Step 1: Topic identification
        if "identify_topics" in steps:
            topic_prompt = f"""
            Step 1: Identify the main topics and themes in this text.
            List 3-5 key topics with brief descriptions.
            
            Text: {text}
            
            Main Topics:
            """
            results["topics"] = self.generate(topic_prompt, temperature=0.3)
        
        # Step 2: Key point extraction
        if "extract_key_points" in steps:
            points_prompt = f"""
            Step 2: For each topic identified below, extract the key points from the text.
            
            Topics: {results.get('topics', 'Not identified')}
            Text: {text}
            
            Key Points by Topic:
            """
            results["key_points"] = self.generate(points_prompt, temperature=0.2)
        
        # Step 3: Synthesis
        if "synthesize" in steps:
            synthesis_prompt = f"""
            Step 3: Synthesize the following analysis into a coherent summary.
            
            Topics: {results.get('topics', '')}
            Key Points: {results.get('key_points', '')}
            
            Create a flowing summary that integrates all important information:
            
            Final Summary:
            """
            results["final_summary"] = self.generate(synthesis_prompt, temperature=0.4)
        
        return results
    
    def quality_control_pipeline(self, text, summary):
        """Apply quality control checks and improvements"""
        # Check factuality
        factuality = self.generate(
            self.quality_checks["factuality_check"].format(
                source=text, summary=summary
            ), temperature=0.1
        )
        
        # Improve coherence if needed
        improved_summary = self.generate(
            self.quality_checks["coherence_improvement"].format(
                summary=summary
            ), temperature=0.3
        )
        
        return {
            "original_summary": summary,
            "factuality_check": factuality,
            "improved_summary": improved_summary
        }

In [ ]:
with open("articles/article.txt", 'r', encoding='utf-8') as f:
    article_text = f.read()
with open("articles/complex_article.txt", 'r', encoding='utf-8') as f:
    complex_article_text = f.read()

def demonstrate_advanced_prompting():
    """Demonstrate all advanced prompting techniques"""
    
    advanced_prompter = AdvancedSummarizationPrompting()
    
    # Domain-specific summarization
    news_summary = advanced_prompter.generate_domain_summary(
        article_text, 
        domain="news", 
        audience="general public"
    )
    
    # Multi-step summarization
    complex_summary = advanced_prompter.multi_step_summarization(
        complex_article_text
    )
    
    # Quality control
    quality_results = advanced_prompter.quality_control_pipeline(
        article_text, 
        news_summary
    )
    
    return {
        "domain_specific": news_summary,
        "multi_step": complex_summary,
        "quality_controlled": quality_results
    }

advanced_prompting_results = demonstrate_advanced_prompting()

In [ ]:
def experiment_with_prompts():
    """Experiment with different instruction styles"""
    
    instructions = {
        "basic": "Summarize the following text:",
        "concise": "Provide a concise summary of the main points:",
        "detailed": "Write a comprehensive summary covering all key aspects:",
        "bullet": "Summarize the key points as bullet points:",
        "executive": "Write an executive summary focusing on implications and next steps:"
    }
    
    results = {}
    
    for style, instruction in instructions.items():
        print(f"\n--- {style.upper()} STYLE ---")
        summary = abstractive_summarizer.zero_shot_summarize(article, instruction)
        results[style] = summary
        print(f"Instruction: {instruction}")
        print(f"Summary: {summary}")
        print(f"Length: {len(summary.split())} words")
    
    return results

instruction_results = experiment_with_prompts()

## **2.3 Evaluation Metrics for Summarization**

How do we know if our summaries are good? Let's implement some common evaluation metrics:

### ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
- Measures overlap between machine-generated summary and reference summary
- ROUGE-N: N-gram recall
- ROUGE-L: Longest Common Subsequence 

<img src='images/Rouge L.jpeg' width=700>

LCS is the longest set of ordered tokens that occurs in both sequences (Ref, Gen)

### BLEU (Bilingual Evaluation Understudy)
- Originally designed for translation, but used for summarization
- Precision-focused (how many generated n-grams appear in reference)

### BERTScore
- Uses contextual embeddings to compute similarity
- Better semantic understanding than n-gram methods

### Human Evaluation Dimensions
- **Relevance**: How well does the summary capture the main points?
- **Coherence**: Does it flow logically?
- **Fluency**: Is it grammatically correct?
- **Factuality**: Does it contain errors or hallucinations?
- **Accuracy**: Does the summary accurately represent the original content?
- **Readability**: Is the summary well-written and easy to understand?

In [ ]:
# Implement custom ROUGE socre
from collections import Counter

# Reference summary for comparison
with open("articles/reference summary.txt", 'r', encoding='utf-8') as f:
    reference_summary = f.read()


def calculate_rouge_n(reference_summary, generated_summary, n):
    """Calculate ROUGE-N score"""
    # Tokenize into words
    ref_tokens = word_tokenize(reference_summary.lower())
    cand_tokens = word_tokenize(generated_summary.lower())

    # Generate n-grams
    ref_ngrams = list(zip(*[ref_tokens[i:] for i in range(n)]))
    cand_ngrams = list(zip(*[cand_tokens[i:] for i in range(n)]))
    
    # Count ngrams
    ref_counter = Counter(ref_ngrams)
    cand_counter = Counter(cand_ngrams)
    
    # Count matches
    overlap = sum((ref_counter & cand_counter).values()) # intersection of the two counters

    # Calculate precision and recall
    precision = overlap / max(1, sum(cand_counter.values()))
    recall = overlap / max(1, sum(ref_counter.values()))
    
    # Calculate F1 score
    f1 = 2 * precision * recall / max(1, precision + recall)
    return f1

def calculate_rouge_l(reference_summary, generated_summary):
    """Calculate ROUGE-L (Longest Common Subsequence)"""
    
    def lcs_length(X, Y):
        m, n = len(X), len(Y)
        L = [[0] * (n + 1) for _ in range(m + 1)]
        
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i-1] == Y[j-1]:
                    L[i][j] = L[i-1][j-1] + 1
                else:
                    L[i][j] = max(L[i-1][j], L[i][j-1])
        
        return L[m][n]
    
    # Tokenize
    ref_tokens = word_tokenize(reference_summary.lower())
    gen_tokens = word_tokenize(generated_summary.lower())
    
    # Calculate LCS
    lcs_len = lcs_length(ref_tokens, gen_tokens)
    
    # Calculate precision, recall, F1
    precision = lcs_len / max(1, len(gen_tokens))
    recall = lcs_len / max(1, len(ref_tokens))
    f1 = 2 * precision * recall / max(1, precision + recall)
    
    return f1

In [12]:
# Let's create a simple function to evaluate our summaries for ROUGE-1 and ROUGE-2
def evaluate_summary(reference, candidate):
    """Evaluate a summary using multiple metrics"""
    scores = {
        'ROUGE-1': calculate_rouge_n(reference, candidate, 1),
        'ROUGE-2': calculate_rouge_n(reference, candidate, 2),
    }

    # Add readability metric: average words per sentence
    cand_sentences = sent_tokenize(candidate)
    avg_sentence_length = len(word_tokenize(candidate)) / max(1, len(cand_sentences))
    scores['Avg Words/Sentence'] = avg_sentence_length
    
    return scores

# Evaluate our extractive summary against the reference
evaluation_scores = evaluate_summary(reference_summary, generated_summary)

print("Evaluation Results:")
for metric, score in evaluation_scores.items():
    print(f"{metric}: {score:.4f}")

# **Part III - Advanced Applications**

## **3.1 Complex Document Handling**



In [ ]:
class LongDocumentSummarizer(AbstractiveSummarizer):
    """Specialized summarizer for handling long documents"""
    
    def chunk_text(self, text, chunk_size=1000, overlap=100):
        """Split text into overlapping chunks"""
        words = text.split()
        chunks = []
        
        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)
            
            if i + chunk_size >= len(words):
                break
        
        return chunks
    
    def recursive_summarize(self, text, target_length=200, max_iterations=3):
        """Recursively summarize long text"""
        
        print(f"📄 Starting recursive summarization...")
        print(f"Original length: {len(text.split())} words")
        print(f"Target length: {target_length} words")
        
        current_text = text
        iteration = 0
        
        while len(current_text.split()) > target_length and iteration < max_iterations:
            iteration += 1
            print(f"\n🔄 Iteration {iteration}:")
            
            # If text is still very long, chunk it first
            if len(current_text.split()) > 2000:
                print("  📝 Chunking large text...")
                chunks = self.chunk_text(current_text, chunk_size=800, overlap=50)
                print(f"  Created {len(chunks)} chunks")
                
                # Summarize each chunk
                chunk_summaries = []
                for i, chunk in enumerate(chunks):
                    print(f"  Processing chunk {i+1}/{len(chunks)}...")
                    chunk_summary = self.zero_shot_summarize(
                        chunk, 
                        "Summarize the key points from this text section:"
                    )
                    chunk_summaries.append(chunk_summary)
                
                # Combine chunk summaries
                current_text = ' '.join(chunk_summaries)
                print(f"  Combined chunks: {len(current_text.split())} words")
            
            # Final summarization pass
            if len(current_text.split()) > target_length:
                print("  📋 Final summarization pass...")
                current_text = self.zero_shot_summarize(
                    current_text,
                    f"Create a comprehensive summary of approximately {target_length} words:"
                )
                print(f"  Result: {len(current_text.split())} words")
        
        print(f"\n✅ Final summary: {len(current_text.split())} words")
        return current_text
    
    def hierarchical_summarize(self, text, levels=["detailed", "medium", "brief"]):
        """Create multiple summary levels"""
        
        level_configs = {
            "detailed": {"length": 400, "instruction": "Provide a detailed summary covering all major points:"},
            "medium": {"length": 200, "instruction": "Create a balanced summary of key points:"},
            "brief": {"length": 100, "instruction": "Write a concise summary of main ideas:"},
            "executive": {"length": 50, "instruction": "Provide an executive summary in 2-3 sentences:"}
        }
        
        summaries = {}
        
        for level in levels:
            if level in level_configs:
                config = level_configs[level]
                print(f"\n📊 Creating {level} summary (target: ~{config['length']} words)...")
                
                summary = self.recursive_summarize(text, target_length=config['length'])
                summaries[level] = summary
                
                print(f"✅ {level.capitalize()} summary ({len(summary.split())} words):")
                print(summary[:200] + "..." if len(summary) > 200 else summary)
        
        return summaries

# Initialize long document summarizer
long_doc_summarizer = LongDocumentSummarizer()

In [ ]:
# Test recursive summarization
print("\n🔄 Testing Recursive Summarization:")
recursive_result = long_doc_summarizer.recursive_summarize(long_document, target_length=150)

# Test hierarchical summarization
print("\n📊 Testing Hierarchical Summarization:")
with open("articles/long_document.txt", 'r', encoding='utf-8') as f:
    long_document = f.read()
    
hierarchical_results = long_doc_summarizer.hierarchical_summarize(
    long_document, 
    levels=["detailed", "medium", "brief"]
)

## **3.2 Multi-Document Summarization**


In [ ]:
class MultiDocumentSummarizer(AbstractiveSummarizer):
    """Summarizer for handling multiple related documents"""
    
    def comparative_summarize(self, documents, document_labels=None):
        """Compare and contrast multiple documents"""
        
        if document_labels is None:
            document_labels = [f"Document {i+1}" for i in range(len(documents))]
        
        # Create comparative prompt
        comparative_prompt = """
        I need to analyze and compare multiple documents on related topics. 
        Please provide a comparative summary that:
        1. Identifies common themes across documents
        2. Highlights key differences in perspectives or findings
        3. Synthesizes the most important information
        4. Notes any contradictions or conflicting information
        
        Documents to compare:
        """
        
        for label, doc in zip(document_labels, documents):
            comparative_prompt += f"\n\n{label}:\n{doc}"
        
        comparative_prompt += "\n\nComparative Summary:"
        
        return self.generate(comparative_prompt, max_tokens=500)
    
    def synthesis_summarize(self, documents, focus_question=None):
        """Synthesize information from multiple sources"""
        
        if focus_question is None:
            focus_question = "What are the key insights when considering all sources together?"
        
        synthesis_prompt = f"""
        Synthesize information from the following sources to answer: {focus_question}
        
        Please create a synthesis that:
        - Integrates information from all sources
        - Identifies patterns and trends
        - Resolves or notes conflicting information
        - Provides a coherent unified perspective
        
        Sources:
        """
        
        for i, doc in enumerate(documents, 1):
            synthesis_prompt += f"\n\nSource {i}:\n{doc}"
        
        synthesis_prompt += f"\n\nSynthesis addressing '{focus_question}':"
        
        return self.generate(synthesis_prompt, max_tokens=500)

# Initialize multi-document summarizer
multi_doc_summarizer = MultiDocumentSummarizer()

###  Multi-Document Exercise

In [ ]:
def test_multi_document_summarization():
    """Test multi-document summarization with different perspectives"""
    
    # Three documents with different perspectives on remote work
    documents = [
        """
        Study A: Remote Work Productivity Analysis
        A comprehensive study of 5,000 employees across 50 companies found that remote workers are 13% more productive than their office counterparts. The research, conducted over 18 months, measured productivity through completed tasks, project deadlines met, and output quality scores. Remote workers reported higher job satisfaction (8.2/10 vs 7.1/10) and better work-life balance. However, the study noted challenges in spontaneous collaboration and team building. Companies with strong digital infrastructure saw the highest productivity gains, while those with poor remote work policies experienced productivity declines.
        """,
        """
        Study B: Remote Work Challenges and Solutions
        Research focusing on management perspectives reveals significant challenges in remote work implementation. Surveys of 1,200 managers indicate concerns about employee oversight (67%), team cohesion (58%), and maintaining company culture (52%). The study found that productivity varies significantly by role type, with creative and collaborative roles showing 8% decreased output while individual contributor roles improved by 12%. Communication frequency increased by 35% in remote teams, but decision-making speed decreased by 23%. Companies investing in management training for remote leadership saw better outcomes.
        """,
        """
        Study C: Economic Impact of Remote Work
        Economic analysis of remote work trends shows substantial cost savings for both employers and employees. Companies report average savings of $11,000 per remote employee annually through reduced office space, utilities, and facility costs. Employees save an average of $4,000 yearly on commuting, work clothing, and meals. However, the analysis reveals increased spending on home office equipment and higher utility bills for workers. The real estate market has been significantly impacted, with commercial office space demand down 30% in major cities while residential markets in suburban areas have seen 15% price increases. The shift has created an estimated $1.2 trillion economic redistribution.
        """
    ]
    
    labels = ["Productivity Study", "Management Perspective", "Economic Analysis"]
    
    print("📑 COMPARATIVE ANALYSIS:")
    print("=" * 60)
    comparative_result = multi_doc_summarizer.comparative_summarize(documents, labels)
    print(comparative_result)
    
    print("\n🔬 SYNTHESIS FOR POLICY MAKERS:")
    print("=" * 60)
    synthesis_result = multi_doc_summarizer.synthesis_summarize(
        documents, 
        "What policy recommendations emerge from considering all three studies?"
    )
    print(synthesis_result)
    
    return comparative_result, synthesis_result

comparative_analysis, synthesis_analysis = test_multi_document_summarization()

## **3.3 Multimodal Summarization**


Multimodal summarization involves generating concise text that captures information from:
- Text documents
- Images
- Tables and charts
- Audio recordings
- Video content

## Approaches to Multimodal Summarization:

1. **Pipeline Approach**: Process each modality separately, then combine
2. **Unified Models**: Use multimodal models (like CLIP or GPT-4) that understand multiple modalities

In [ ]:
# Implementing a multimodal summarizer for text + image data 

import requests
import base64
import os
import json
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

class MultimodalSummarizer:
    """Multimodal summarizer using Llama-4 Vision capabilities"""
    
    def __init__(self, api_key=None):
        """Initialize the multimodal summarizer with an OPENROUTER API key"""
        # Get API key from environment variable if not provided
        self.api_key = api_key or os.environ.get("OPENROUTER_API_KEY", "")
        if not self.api_key:
            print("Warning: No OPENROUTER API key provided. Please set your OPENROUTER_API_KEY.")
        
        self.api_url = "https://openrouter.ai/api/v1"
        self.model = "meta-llama/llama-4-maverick:free"
        
    def encode_image(self, image_path):
        """Encode an image to base64 for API submission"""
        # Check if it's a URL or local path
        if image_path.startswith(('http://', 'https://')):
            response = requests.get(image_path)
            image = Image.open(BytesIO(response.content))
            buffered = BytesIO()
            image.save(buffered, format="JPEG")
            return base64.b64encode(buffered.getvalue()).decode('utf-8')
        else:
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
    
    def create_payload(self, text, image_paths, max_tokens=500):
        """Create the API payload with text and images"""
        messages = [
            {
                "role": "system",
                "content": "You are a helpful assistant that creates concise summaries from text and images."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Please create a comprehensive summary that combines information from the following text and images. Focus on integrating visual information with the text content.\n\nTEXT: {text}"}
                ]
            }
        ]
        
        # Add images to the content
        for img_path in image_paths:
            try:
                base64_image = self.encode_image(img_path)
                messages[1]["content"].append(
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "high"
                        }
                    }
                )
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
        
        return {
            "model": self.model,
            "messages": messages,
            "max_tokens": max_tokens
        }
    
    def summarize_multimodal(self, text, image_paths, max_tokens=500):
        """Generate a summary from text and images using GPT-4"""
        if not self.api_key:
            return {"error": "No API key provided. Please set your OPENROUTER_API_KEY."}
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        payload = self.create_payload(text, image_paths, max_tokens)
        
        try:
            print("Making API call to model...")

            response = requests.post(self.api_url, headers=headers, data=payload)
            result = response.json()
            
            return {
                'text_source': text,
                'image_paths': image_paths,
                'combined_summary': result
            }
            
        except Exception as e:
            return {"error": str(e)}
    
    def display_images(self, image_paths):
        """Display the images used in the multimodal summary"""
        num_images = len(image_paths)
        
        if num_images == 0:
            return
        
        fig, axes = plt.subplots(1, num_images, figsize=(5*num_images, 5))
        
        if num_images == 1:
            axes = [axes]  
            
        for i, img_path in enumerate(image_paths):
            try:
                # Handle both URLs and local paths
                if img_path.startswith(('http://', 'https://')):
                    response = requests.get(img_path)
                    img = Image.open(BytesIO(response.content))
                else:
                    img = Image.open(img_path)
                
                axes[i].imshow(img)
                axes[i].set_title(f"Image {i+1}")
                axes[i].axis('off')
            except Exception as e:
                axes[i].text(0.5, 0.5, f"Error loading image: {e}", 
                             ha='center', va='center', transform=axes[i].transAxes)
        
        plt.tight_layout()
        plt.savefig('multimodal_input.png')
        plt.close()
        
        from IPython.display import Image
        return Image('multimodal_input.png')

In [ ]:
image_paths = [
    # "images/image1.jpg",
    # "images/image2.jpg",
    # "images/image3.png"
]

multimodal_summarizer = MultimodalSummarizer()

# Generate a multimodal summary
multimodal_result = multimodal_summarizer.summarize_multimodal(
    #article,
    image_paths,
    max_tokens=300
)

### Practical Exercise: Building Your Custom Summarization System

Now it's your turn to build a complete summarization system by combining techniques we've explored.


## Project Ideas:
1. **News Summarizer Bot**: Create a system that retrieves and summarizes news articles on specific topics
2. **Meeting Minutes Generator**: Transcribe and summarize meeting audio recordings
3. **Research Paper Summarizer**: Generate summaries of academic papers with focus on methodology and results
4. **Medical Conversation Summarizer**: Summarize doctor-patient conversations, creating dual summaries (technical for doctors, simplified for patients)
5. **EHR Summarizer**: Create a system that generates longitudinal patient summaries from fragmented electronic health records, retrieving and synthesizing information across multiple visits, lab results, and clinical notes


### Code and Libraries:
- [Transformers Documentation](https://huggingface.co/docs/transformers/index)
- [Text Generation Parameters](https://huggingface.co/blog/mlabonne/decoding-strategies)


## Useful Resources:

### Datasets:
- [CNN/Daily Mail Dataset](https://huggingface.co/datasets/cnn_dailymail)
- [XSum Dataset](https://huggingface.co/datasets/xsum)
- [Multi-News](https://huggingface.co/datasets/multi_news)
- [BBC News Summary Dataset](https://www.kaggle.com/datasets/pariza/bbc-news-summary)

### Evaluation Tools:
- [ROUGE Implementation in Python](https://github.com/google-research/google-research/tree/master/rouge)



Utils

In [ ]:
# Transcribing audio files
import whisper

def transcribe_meeting(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result["text"]

# download audio from youtube
import yt_dlp
# pip install yt-dlp
def download_audio_yt_dlp(url: str, output_path: str = "downloads") -> str:
    """
    Downloads audio from a YouTube video using yt-dlp.

    Args:
        url (str): YouTube video URL.
        output_path (str): Directory to save the audio.

    Returns:
        str: Path to the downloaded file.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        return f"{output_path}/{info['title']}.mp3"


# **Facilitator(s) Details**

**Facilitator(s):**

*   Name: Nana Sam Yeboah                       
*   Email: nanayeb34@gmail.com
*   LinkedIn: [Nana Sam Yeboah](https://www.linkedin.com/in/nana-sam-yeboah-0b664484)

# 

*   Name: Audrey Eyram Agbeve
*   Email: audreyagbeve02@gmail.com
*   LinkedIn: [Audrey (Eyram) Agbeve](https://www.linkedin.com/in/audreyagbeve02/)

### Please rate this Tutorial

<img src="images/Day1_feedback.png" height=500 width=500  >